# 열쇠

[baekjoon의 '열쇠' 링크](https://www.acmicpc.net/problem/9328)

### 문제   
상근이는 1층 빌딩에 침입해 매우 중요한 문서를 훔쳐오려고 한다. 상근이가 가지고 있는 평면도에는 문서의 위치가 모두 나타나 있다. 빌딩의 문은 모두 잠겨있기 때문에, 문을 열려면 열쇠가 필요하다. 상근이는 일부 열쇠를 이미 가지고 있고, 일부 열쇠는 빌딩의 바닥에 놓여져 있다.  
  
상근이가 훔칠 수 있는 문서의 최대 개수를 구하는 프로그램을 작성하시오.  
  
### 입력  
첫째 줄에 테스트 케이스의 개수가 주어진다. 테스트 케이스의 수는 100개를 넘지 않는다.  
  
각 테스트 케이스의 첫째 줄에는 지도의 높이와 너비 h와 w (2 ≤ h, w ≤ 100)가 주어진다. 다음 h개 줄에는 빌딩을 나타내는 w개의 문자가 주어지며, 각 문자는 다음 중 하나이다.  
  
- '.'는 빈 공간을 나타낸다.  
- '\*'는 벽을 나타내며, 상근이는 벽을 통과할 수 없다.  
- '$'는 상근이가 훔쳐야하는 문서이다.   
- 알파벳 대문자는 문을 나타낸다.  
- 벳 소문자는 열쇠를 나타내며, 그 문자의 대문자인 모든 문을 열 수 있다.  
  
마지막 줄에는 상근이가 이미 가지고 있는 열쇠가 공백없이 주어진다. 만약, 열쇠를 하나도 가지고 있지 않는 경우에는 "0"이 주어진다.  
  
상근이는 처음에는 빌딩의 밖에 있으며, 빌딩 가장자리의 빈 공간이나 문을 통해 빌딩 안팎을 드나들 수 있다. 각각의 문에 대해서, 그 문을 열 수 있는 열쇠의 개수는 0개, 1개, 또는 그 이상이고, 각각의 열쇠에 대해서, 그 열쇠로 열 수 있는 문의 개수도 0개, 1개, 또는 그 이상이다. 열쇠는 여러 번 사용할 수 있다.  
  
### 출력  
각 테스트 케이스 마다, 상근이가 훔칠 수 있는 문서의 최대 개수를 출력한다.  

In [1]:
from queue import Queue
from copy import deepcopy

def simulation(h, w, board, moved, s_keys, s_docs, s_y, s_x):
    checked = [[(-1, -1) for _ in range(w)] for _ in range(h)]
    direction = [(0,1), (1,0), (0,-1), (-1,0)]
    
    if board[s_y][s_x].islower():
        s_keys.add(board[s_y][s_x])
    elif board[s_y][s_x] == '$':
        s_docs.add((s_y, s_x))
    max_docs = len(s_docs)
    max_keys = len(s_keys)
    checked[s_y][s_x] = (len(s_keys), len(s_docs))
    q = Queue()
    q.put([s_docs, s_keys, s_y, s_x])
    while not q.empty():
        docs, keys, y, x = q.get()
        docs_num, keys_num = len(docs), len(keys)
        moved[y][x] = True
        
        if keys_num > max_keys:
            s_keys = keys
        if docs_num > max_docs:
            s_docs = docs
            
        for add_y, add_x in direction:
            if 0 <= add_y+y < h and 0 <= add_x+x < w:
                if board[add_y+y][add_x+x] == '*':
                    continue
                elif (board[add_y+y][add_x+x] == '.' or board[add_y+y][add_x+x].lower() in keys) and checked[add_y+y][add_x+x] < (keys_num, docs_num):
                    checked[add_y+y][add_x+x] = (keys_num, docs_num)
                    q.put([docs, keys, add_y+y, add_x+x])
                elif board[add_y+y][add_x+x].islower() and checked[add_y+y][add_x+x] < (keys_num+1, docs_num):
                    copy_keys = deepcopy(keys)
                    copy_keys.add(board[add_y+y][add_x+x])
                    checked[add_y+y][add_x+x] = (keys_num+1, docs_num)
                    q.put([docs, copy_keys, add_y+y, add_x+x])
                elif board[add_y+y][add_x+x] == '$' and not (add_y+y, add_x+x) in docs and checked[add_y+y][add_x+x] < (keys_num, docs_num+1):
                    copy_docs = deepcopy(docs)
                    copy_docs.add((add_y+y, add_x+x))
                    checked[add_y+y][add_x+x] = (keys_num, docs_num+1)
                    q.put([copy_docs, keys, add_y+y, add_x+x])
    return s_keys, s_docs

print("Input")
T = int(input()) # 테스트케이스 개수
answer = [] # 답
for _ in range(T):
    h, w = tuple(int(x) for x in input().split()) # 높이, 너비
    doc_num = 0 # 
    board = [] # 보드
    for y in range(h):
        row = list(input())
        board.append(row)
    keys = set(input()) # 갖고있는 키
    if keys == {'0'}:
        keys = set()
    docs = set() # 얻은 문서의 위치
    changed = True # 변화 여부
    while changed:
        moved = [[False for _ in range(w)] for _ in range(h)] # 해당 위치에 탐색했는지 여부
        changed = False # 변화 여부
        for y in range(h):
            if y == 0 or y == h-1:
                for x in range(w):
                    if not moved[y][x] and board[y][x] != '*': # 아직 탐색하지 않은 벽을 제외한 위치
                        if board[y][x].isupper() and not board[y][x].lower() in keys:
                            continue
                        # 열쇠를 지니지 않은 문은 제외한다.
                        keys_added, docs_added = simulation(h, w, board, moved, keys, docs, y, x)
                        # 시뮬레이션 진행
                        if keys != keys_added:
                            keys = keys_added
                            changed = True
                        if docs != docs_added:
                            docs = docs_added
                            changed = True
                        # 키와 문서들을 갱신해준다.
            else:
                for x in [0, w-1]:
                    if not moved[y][x] and board[y][x] != '*': # 아직 탐색하지 않은 벽을 제외한 위치
                        if board[y][x].isupper() and not board[y][x].lower() in keys:
                            continue
                        # 열쇠를 지니지 않은 문은 제외한다.
                        keys_added, docs_added = simulation(h, w, board, moved, keys, docs, y, x)
                        # 시뮬레이션 진행
                        if keys != keys_added:
                            keys = keys_added
                            changed = True
                        if docs != docs_added:
                            docs = docs_added
                            changed = True
                        # 키와 문서들을 갱신해준다.
        # 넘나들 수 있는 가장자리 위치로 빌딩을 들어갈 수 있을 경우 시뮬레이션을 진행한다.
        # 그리고 얻는 키와 문서가 없을 때까지 시뮬레이션을 계속 진행
    answer.append(len(docs)) # 탐색이 끝나고 문서의 개수를 답에 저장

print("\nOutput")
for a in answer:
    print(a) # 답 출력

Input
3
5 17
*****************
.............**$*
*B*A*P*C**X*Y*.X.
*y*x*a*p**$*$**$*
*****************
cz
5 11
*.*********
*...*...*x*
*X*.*.*.*.*
*$*...*...*
***********
0
7 7
*ABCDE*
X.....F
W.$$$.G
V.$$$.H
U.$$$.J
T.....K
*SQPML*
irony

Output
3
1
0
